In [ ]:
!git clone --recursive https://github.com/perso00k/aml-2025-mistake-detection.git code

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def import_features(source_dir, dest_dir):
  import shutil
  import os
  from tqdm import tqdm

  os.makedirs(dest_dir, exist_ok=True)

  print(f"Inizio copia da {source_dir} a {dest_dir}...")

  # Ottieni lista file .npz (o .pt se non li hai ancora convertiti)
  files = [f for f in os.listdir(source_dir) if f.endswith('.npz') or f.endswith('.pt')]

  if len(files) == 0:
      print("ATTENZIONE: Nessun file .npz o .pt trovato! Controlla il percorso source_dir.")
  else:
      for filename in tqdm(files):
          src = os.path.join(source_dir, filename)
          dst = os.path.join(dest_dir, filename)

          # Copia solo se non esiste già (così se rilanci non perde tempo)
          if not os.path.exists(dst):
              shutil.copy2(src, dst)

      print("Copia completata.")

In [ ]:
source_dir_video = "/content/drive/MyDrive/AML_Project_resized_ds/features_PE_npz_1s_1s_final"
dest_dir_video = "code/data/video/perception_encoder"
import_features(source_dir_video, dest_dir_video)

source_dir_text = "/content/drive/MyDrive/AML_Project_resized_ds/text_feature_PE"
dest_dir_text = "code/data/graphs"
import_features(source_dir_text, dest_dir_text)

# SUBSTEP 1

In [ ]:
!python /content/code/step_localization/step_localizator_dtw.py

# Valutazione

In [ ]:
import os
import json
import numpy as np
from tqdm import tqdm

# --- CONFIGURAZIONE ---
PREDICTIONS_DIR = "/content/code/data/localized_features"
# Questo è il file delle VERITÀ (Ground Truth) fornito dai prof/dataset
GROUND_TRUTH_FILE = "/content/code/annotations/annotation_json/step_annotations.json"

def compute_frame_accuracy(pred_segments, gt_steps, video_duration):
    """
    Calcola la Mean Over Frame (MoF) accuracy.
    Confronta secondo per secondo se l'ID dello step predetto coincide con quello vero.
    """
    # Arrotondiamo la durata per creare l'array dei secondi
    duration_int = int(np.ceil(video_duration))
    
    # 0 = Background/Nessuno step (spesso ignorato o contato come errore se non annotato)
    gt_array = np.zeros(duration_int, dtype=int) 
    pred_array = np.zeros(duration_int, dtype=int)
    
    # Riempiamo l'array con la Verità (Ground Truth)
    for step in gt_steps:
        sid = int(step['step_id'])
        s = max(0, int(np.floor(step['start_time'])))
        e = min(duration_int, int(np.ceil(step['end_time'])))
        gt_array[s:e] = sid
        
    # Riempiamo l'array con le nostre Predizioni
    for seg in pred_segments:
        # seg è [start, end, step_id]
        sid = int(seg[2]) 
        s = max(0, int(np.floor(seg[0])))
        e = min(duration_int, int(np.ceil(seg[1])))
        pred_array[s:e] = sid
        
    # Calcoliamo l'accuracy solo dove c'è un'annotazione valida nel GT
    # (escludiamo il 'silenzio' background se non è annotato esplicitamente)
    mask = gt_array != 0
    
    if np.sum(mask) == 0:
        return 0.0 # Nessun ground truth valido per questo video
        
    matches = (gt_array[mask] == pred_array[mask])
    accuracy = np.mean(matches)
    
    return accuracy

def main():
    if not os.path.exists(GROUND_TRUTH_FILE):
        print(f"❌ Errore: Non trovo il file {GROUND_TRUTH_FILE}")
        print("Assicurati di aver caricato il file step_annotations.json nella cartella corretta.")
        return

    with open(GROUND_TRUTH_FILE, 'r') as f:
        gt_data = json.load(f)
        
    accuracies = []
    processed_count = 0
    missing_count = 0
    
    print("📊 Valutazione Accuratezza Localizzazione (Zero-Shot vs Ground Truth)...")
    
    for video_id, info in tqdm(gt_data.items()):
        # Cerca il file di predizione corrispondente (generato dal tuo script localize)
        pred_path = os.path.join(PREDICTIONS_DIR, f"{video_id}.npz")
        
        # Se non abbiamo processato questo video (es. mancavano feature), saltiamo
        if not os.path.exists(pred_path):
            missing_count += 1
            continue
            
        try:
            # Carica predizione
            data = np.load(pred_path)
            if 'segments' not in data: 
                continue
            
            pred_segments = data['segments'] # Array Nx3: [start, end, step_id]
            gt_steps = info['steps']
            
            # Calcola la durata massima per dimensionare gli array
            if len(pred_segments) > 0:
                max_pred = np.max(pred_segments[:, 1])
            else: max_pred = 0
            
            max_gt = 0
            if gt_steps:
                max_gt = max([s['end_time'] for s in gt_steps])
                
            duration = max(max_pred, max_gt) + 5 # +5 secondi di margine
            
            # Calcola score per questo video
            acc = compute_frame_accuracy(pred_segments, gt_steps, duration)
            accuracies.append(acc)
            processed_count += 1
            
        except Exception as e:
            # print(f"Errore su {video_id}: {e}")
            pass
        
    if processed_count == 0:
        print("❌ Nessun video valutato. Hai eseguito localize_steps_dtw.py?")
        print(f"Video mancanti: {missing_count}")
    else:
        mean_acc = np.mean(accuracies) * 100
        print(f"\n✅ Valutazione Completata su {processed_count} video.")
        print(f"🚫 Video mancanti/saltati: {missing_count}")
        print(f"📈 Mean Frame Accuracy (MoF): {mean_acc:.2f}%")
        print("------------------------------------------------")
        print("INTERPRETAZIONE:")
        print(" < 15%: Localizzazione scarsa")
        print(" 20-30%: Buon risultato per Zero-Shot (senza training)")
        print(" > 40%: Risultato eccellente")

if __name__ == "__main__":
    main()